## Q3.1 

#### Packages and functions
convert function to convert the string to vector of word counts.

cross_validation function to split train_data and train_labels into k parts.

cv_modeling function perform multi-perceptron classifier on dataset with k_folds.

In [1]:
import tensorflow as tf
from tensorflow import keras
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import random
import glob
import matplotlib.pyplot as plt
import pandas as pd
import re, string
from collections import Counter
import numpy as np
from statistics import mean 

In [2]:
#convert function to convert the string to vector of word counts
def convert(string):
    l = []
    s = string.split(' ')
    for word in lexicon:
        c=0
        for i in range(len(s)):
            if s[i]==word:
                c+=1
        l.append(c)
    return(l)

In [3]:
#cross_validation function to split train_data and train_labels into k parts
#output format:tuple (data_list, label_list)
#format of data and label list: array[fold_1_train, fold_2_train,...], array[fold_1_label, fold_2_label,... )]
#format of fold: array [review1, review2,...], array [label1, label2,...]
def cross_validation(k, x, y):
    sample_size = round(len(y)/k)+1
    index_list = range(len(y))
    d_list = []
    data_list = []
    label_list = []
    for i in range(k-1):
        d_list.append(random.sample(index_list, sample_size))
        data_list.append(x[d_list[i]])
        label_list.append(y[d_list[i]])
        index_list = [w for w in index_list if not w in d_list[i]]
    d_list.append(index_list)
    data_list.append(x[d_list[i+1]])
    label_list.append(y[d_list[i+1]])
    return((np.array(data_list),np.array(label_list)))

In [4]:
#cv_modeling function perform multi-perceptron classifier on dataset with k_folds
#return the average accuracy of k interation
def cv_modeling(x,y,d,hi_1_nodes, activation_1):
    acc = []
    k = len(y)
    for i in range(k):
        train_idx = [w for w in range(k) if w!=i]
        train = np.concatenate(x[train_idx])
        labels = np.concatenate(y[train_idx])
        inputs = tf.keras.Input(shape=(d,))
        hi_1 = tf.keras.layers.Dense(hi_1_nodes, activation=activation_1)(inputs)
        hi_2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(hi_1)
        outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hi_2)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
        history_dict = model.fit(train,
                    labels,
                    validation_data=(x[i],y[i]),
                    verbose=1).history
        acc.append(history_dict['val_acc'])
    m = sum(sum(acc,[]))/len(acc)
    return(m)

#### Data Preprocessing
Read files

In [5]:
#list all the input files
read_files_pos = glob.glob("train\\pos\\*.txt")
read_files_neg = glob.glob("train\\neg\\*.txt")
read_files = read_files_pos+read_files_neg

In [6]:
#input files into train and remove punctuation and multiple spaces
re_p = re.compile('[%s]' % re.escape(string.punctuation))
train = []
train_labels = []
for file in read_files:
    train.append(re.sub(r' {2,}',' ', re.sub(r'[\s]',' ',re_p.sub(' ', open(file,'r').read()))))
    train_labels.append(1 if file in read_files_pos else 0)

Generate features

In [27]:
#create a list including all unigrams
unigram = []
for i in range(len(train)):
    all_words = word_tokenize(train[i].lower())
    unigram += list(all_words)
#select the top 10000 frequent words and remove stop words from them
#9855 unigrams remained
lexicon_all= [word for word, word_count in Counter(unigram).most_common(10000)]
stop_words = set(stopwords.words('english'))
lexicon = [w for w in lexicon_all if not w in stop_words]

In [9]:
#convert the train files of string to train_data vector of word counts 
#This step takes around 5 minutes
train_data = list(map(convert, train))
#transform the train data (x) and train labels(y) to array.
train_data = np.array(train_data)
train_labels = np.array(train_labels,dtype='int64')

#### modeling part
model result saved in dictionary grid search.

In [11]:
#split train_data and train_labels into k parts
data = cross_validation(10, train_data, train_labels)

In [12]:
list_nodes = [20, 50, 100, 300]
list_act = {'tanh':tf.nn.tanh, 'sigmoid':tf.nn.sigmoid, 'relu':tf.nn.relu}
grid = []
key = []
for i in range(len(list_nodes)):
    for j in list_act:
        grid.append((list_nodes[i],list_act[j]))
        key.append((list_nodes[i],j))
grid_search = dict.fromkeys(set(key))

In [13]:
#Takes 20 mins to run
#perform multi-perceptron classifier on processed dataset with 10 folds for each hyper parameter
grid_search = {}
dim = len(lexicon)
for i in range(len(grid)):
    grid_search[key[i][0],key[i][1]]= cv_modeling(data[0],data[1],dim,grid[i][0],grid[i][1])

Instructions for updating:
Colocations handled automatically by placer.
Train on 1019 samples, validate on 114 samples
Instructions for updating:
Use tf.cast instead.
1019/1019 [==============================] - 1s 680us/sample - loss: 0.5911 - acc: 0.6791 - val_loss: 0.5046 - val_acc: 0.7719
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 454us/sample - loss: 0.5995 - acc: 0.6752 - val_loss: 0.5045 - val_acc: 0.7719
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 462us/sample - loss: 0.5919 - acc: 0.6968 - val_loss: 0.4852 - val_acc: 0.8070
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 511us/sample - loss: 0.6461 - acc: 0.6222 - val_loss: 0.5242 - val_acc: 0.7544
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 609us/sample - loss: 0.6054 - acc: 0.6585 - val_loss: 0.4768 - val_acc: 0.8070
Train on 1

Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 1ms/sample - loss: 0.6373 - acc: 0.6398 - val_loss: 0.5656 - val_acc: 0.8333
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 1ms/sample - loss: 0.6219 - acc: 0.7105 - val_loss: 0.5220 - val_acc: 0.8421
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 1ms/sample - loss: 0.6559 - acc: 0.6349 - val_loss: 0.6168 - val_acc: 0.7456
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 1s 1ms/sample - loss: 0.6031 - acc: 0.6860 - val_loss: 0.5013 - val_acc: 0.8421
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 2s 2ms/sample - loss: 0.6114 - acc: 0.6879 - val_loss: 0.4799 - val_acc: 0.8421
Train on 1026 samples, validate on 107 samples
1026/1026 [==============================] - 2s 2ms/sample - loss: 0.6197 - acc: 0.6540 - val

In [14]:
grid_search

{(20, 'tanh'): 0.7668716132640838,
 (20, 'sigmoid'): 0.7416625678539276,
 (20, 'relu'): 0.7512543082237244,
 (50, 'tanh'): 0.8012543082237243,
 (50, 'sigmoid'): 0.793531721830368,
 (50, 'relu'): 0.7916625738143921,
 (100, 'tanh'): 0.8025332033634186,
 (100, 'sigmoid'): 0.7945810914039612,
 (100, 'relu'): 0.8211264133453369,
 (300, 'tanh'): 0.8154041647911072,
 (300, 'sigmoid'): 0.8092638075351715,
 (300, 'relu'): 0.7996720790863037}

From the result, the model with 100 nodes in first hidden layer with activation function relu performs best with an accuracy of 82%.

## Q3.2

In [15]:
#set 100 nodes in first layer, use relu as activation function
#train model on the total train_data
inputs = tf.keras.Input(shape=(dim,))
hi_1 = tf.keras.layers.Dense(100, activation=tf.nn.relu)(inputs)
hi_2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(hi_1)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hi_2)
model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)
model_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history_dict = model_1.fit(train_data,
                    train_labels,
                    verbose=1).history

1133/1133 [==============================] - 3s 2ms/sample - loss: 0.6099 - acc: 0.6717


Accuracy is 67% on the whole training set.

## Q3.3

#### Packages and functions
function word2vec_embedding to generate word2vec representation of the reviews

In [8]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim

In [9]:
# function to generate word2vec representation of the reviews 
# and remove stop words from them
def word2vec_embedding(input_s, vec_pre):
    vec = []
    for i in range(len(input_s)):
        l = [w for w in input_s[i].split(' ') if w in vec_pre and not w in stop_words]
        vec_l = []
        for j in range(len(l)):
            vec_l.append(vec_pre[l[j]])
        vec_l = sum(vec_l)/len(vec_l)
        vec.append(vec_l)
    return vec     

#### Data processing
Read files

In [ ]:
#list all the input files
read_files_pos = glob.glob("train\\pos\\*.txt")
read_files_neg = glob.glob("train\\neg\\*.txt")
read_files = read_files_pos+read_files_neg

In [ ]:
#input files into train and remove punctuation and multiple spaces
re_p = re.compile('[%s]' % re.escape(string.punctuation))
train = []
train_labels = []
for file in read_files:
    train.append(re.sub(r' {2,}',' ', re.sub(r'[\s]',' ',re_p.sub(' ', open(file,'r').read()))))
    train_labels.append(1 if file in read_files_pos else 0)
stop_words = set(stopwords.words('english'))

Generate features

In [10]:
#Unzip the file to word directory
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)

In [13]:
#generate word2vec representation of the reviews 
train_vec = np.array(word2vec_embedding(train, word2vec), dtype='float64')
train_labels = np.array(train_labels,dtype='int64')

#### Modeling part

In [14]:
#split train_data and train_labels into k parts
data_embedding = cross_validation(10, train_vec, train_labels)

In [16]:
list_nodes = [20, 50, 100, 300]
list_act = {'tanh':tf.nn.tanh, 'sigmoid':tf.nn.sigmoid, 'relu':tf.nn.relu}
grid = []
key = []
for i in range(len(list_nodes)):
    for j in list_act:
        grid.append((list_nodes[i],list_act[j]))
        key.append((list_nodes[i],j))
grid_search = dict.fromkeys(set(key))

In [17]:
#Takes 15 mins to run
#perform multi-perceptron classifier on processed dataset with 10 folds for each hyper parameter
grid_search = {}
for i in range(len(grid)):
    grid_search[key[i][0],key[i][1]]= cv_modeling(data_embedding[0],data_embedding[1],300,grid[i][0],grid[i][1])

Instructions for updating:
Colocations handled automatically by placer.
Train on 1019 samples, validate on 114 samples
Instructions for updating:
Use tf.cast instead.
1019/1019 [==============================] - 0s 401us/sample - loss: 0.6928 - acc: 0.5064 - val_loss: 0.6967 - val_acc: 0.4474
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 292us/sample - loss: 0.6930 - acc: 0.5113 - val_loss: 0.6957 - val_acc: 0.3772
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 297us/sample - loss: 0.6907 - acc: 0.5132 - val_loss: 0.6889 - val_acc: 0.6053
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 333us/sample - loss: 0.6913 - acc: 0.5378 - val_loss: 0.6854 - val_acc: 0.6579
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 0s 364us/sample - loss: 0.6930 - acc: 0.5270 - val_loss: 0.6886 - val_acc: 0.7105
Train on 1

Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 2s 2ms/sample - loss: 0.6865 - acc: 0.5800 - val_loss: 0.6949 - val_acc: 0.4649
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 2s 2ms/sample - loss: 0.6896 - acc: 0.5496 - val_loss: 0.6879 - val_acc: 0.4912
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 3s 3ms/sample - loss: 0.6844 - acc: 0.5751 - val_loss: 0.6645 - val_acc: 0.7105
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 3s 3ms/sample - loss: 0.6889 - acc: 0.5368 - val_loss: 0.6850 - val_acc: 0.4912
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 3s 3ms/sample - loss: 0.6951 - acc: 0.5289 - val_loss: 0.6833 - val_acc: 0.4825
Train on 1019 samples, validate on 114 samples
1019/1019 [==============================] - 3s 3ms/sample - loss: 0.6904 - acc: 0.5496 - val

In [18]:
grid_search

{(20, 'tanh'): 0.5735448390245438,
 (20, 'sigmoid'): 0.49209706485271454,
 (20, 'relu'): 0.5272011786699295,
 (50, 'tanh'): 0.5586325615644455,
 (50, 'sigmoid'): 0.4728152215480804,
 (50, 'relu'): 0.574192488193512,
 (100, 'tanh'): 0.5780783712863922,
 (100, 'sigmoid'): 0.5362600445747375,
 (100, 'relu'): 0.5722823560237884,
 (300, 'tanh'): 0.5228152126073837,
 (300, 'sigmoid'): 0.5408591538667679,
 (300, 'relu'): 0.5505082845687866}

From the result, the model with 100 nodes in first hidden layer with activation function tanh performs best with an accuracy of 58%.

#### train model on the whole training data

In [19]:
#set 100 nodes in first layer, use tanh as activation function
#train model on the total train_data
inputs = tf.keras.Input(shape=(300,))
hi_1 = tf.keras.layers.Dense(100, activation=tf.nn.tanh)(inputs)
hi_2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(hi_1)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hi_2)
model_2 = tf.keras.Model(inputs=inputs, outputs=outputs)
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history_dict = model_2.fit(train_vec,
                    train_labels,
                    verbose=1).history

1133/1133 [==============================] - ETA: 0s - loss: 0.6867 - acc: 0.583 - 2s 2ms/sample - loss: 0.6865 - acc: 0.5852


Accuracy is 59% on the whole training set.

## Q3.4
Add one more feature: normalized # of words

#### generate new feature

In [20]:
# compute the length of each each review and normalize them to around 0
length = []
for i in range(len(train)):
    length.append(len(train[i].split(' ')))
avg_length = sum(length)/len(length)
nor_length = []
for i in range(len(length)):
    nor_length.append(length[i]/avg_length-1)

In [21]:
#add the new feature to the word_embedding representation
new_train_vec = []
for i in range(len(train_vec)):
    new_train_vec.append(np.append(train_vec[i], nor_length[i]))
new_train_vec = np.array(new_train_vec)

#### Modeling part

In [23]:
#set 100 nodes in first layer, use tanh as activation function
#train model on the total train_data
inputs = tf.keras.Input(shape=(301,))
hi_1 = tf.keras.layers.Dense(50, activation=tf.nn.relu)(inputs)
hi_2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(hi_1)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hi_2)
model_3 = tf.keras.Model(inputs=inputs, outputs=outputs)
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history_dict = model_3.fit(new_train_vec,
                    train_labels,
                    verbose=1).history

1133/1133 [==============================] - 2s 2ms/sample - loss: 0.6895 - acc: 0.5305


Accuracy is 53% on the whole training set.

## Q3.5
The best model above is the model_1 in Q3.2

#### Model Preparation
rerun this part to get model_1 if restart python

In [5]:
#list all the input files
read_files_pos = glob.glob("train\\pos\\*.txt")
read_files_neg = glob.glob("train\\neg\\*.txt")
read_files = read_files_pos+read_files_neg

In [6]:
#input files into train and remove punctuation and multiple spaces
re_p = re.compile('[%s]' % re.escape(string.punctuation))
train = []
train_labels = []
for file in read_files:
    train.append(re.sub(r' {2,}',' ', re.sub(r'[\s]',' ',re_p.sub(' ', open(file,'r').read()))))
    train_labels.append(1 if file in read_files_pos else 0)

In [7]:
#create a list including all unigrams
unigram = []
for i in range(len(train)):
    all_words = word_tokenize(train[i].lower())
    unigram += list(all_words)
#select the top 10000 frequent words and remove stop words from them
#9855 unigrams remained
lexicon_all= [word for word, word_count in Counter(unigram).most_common(10000)]
stop_words = set(stopwords.words('english'))
lexicon = [w for w in lexicon_all if not w in stop_words]

In [8]:
#convert the train files of string to train_data vector of word counts 
#This step takes around 5 minutes
train_data = list(map(convert, train))
#transform the train data (x) and train labels(y) to array.
train_data = np.array(train_data)
train_labels = np.array(train_labels,dtype='int64')

In [29]:
#set 100 nodes in first layer, use relu as activation function
#train model on the total train_data use 5 epochs
dim = len(lexicon)
inputs = tf.keras.Input(shape=(dim,))
hi_1 = tf.keras.layers.Dense(100, activation=tf.nn.relu)(inputs)
hi_2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)(hi_1)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hi_2)
model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)
model_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history_dict = model_1.fit(train_data,
                    train_labels,
                    epochs=3,
                    verbose=1).history

Epoch 1/3
1133/1133 [==============================] - 1s 843us/sample - loss: 0.6145 - acc: 0.6567
Epoch 2/3
1133/1133 [==============================] - 1s 644us/sample - loss: 0.1753 - acc: 0.9532s - loss: 0.1935 - acc: 
Epoch 3/3
1133/1133 [==============================] - 1s 652us/sample - loss: 0.0278 - acc: 0.9974


#### Data Processing

In [11]:
#input files into test and remove punctuation and multiple spaces
read_test = glob.glob("test\\*.txt")
re_p = re.compile('[%s]' % re.escape(string.punctuation))
test = []
for file in read_test:
    test.append(re.sub(r' {2,}',' ', re.sub(r'[\s]',' ',re_p.sub(' ', open(file,'r').read()))))

In [12]:
#convert the test files of string to test_data vector of word counts 
#This step takes around 5 minutes
test_data = list(map(convert, test))
test_data = np.array(test_data)

#### Model Prediction

In [30]:
pred_result  = model_1.predict(test_data)

In [31]:
pos = []
neg = []
for i in range(len(read_test)):
    if pred_result[i] >= 0.5:
        pos.append(read_test[i].split('\\')[1])
    else:
        neg.append(read_test[i].split('\\')[1])

In [34]:
#output classification result
with open('pos.txt', 'w') as f:
    for i in range(len(pos)):
        f.write('%s\n' % pos[i])
with open('neg.txt', 'w') as f:
    for i in range(len(neg)):
        f.write('%s\n' % neg[i])